In [25]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [26]:
data, location, price, area, time, bill_of_sale, mortgage, repair ={}, [], [], [], [], [], [], []
for i in range(421):
    url = 'https://bina.az/baki/nerimanov/alqi-satqi/menziller/yeni-tikili?page='+str(i+1)
    req = requests.get(url)
    req.encoding = 'utf-8' #class="card_params"
    soup = BeautifulSoup(req.text, "html.parser")
    properties = soup.find_all("div", class_="items-i")
    for item in properties:
        location.append(item.find("div", class_="location").text.strip())
        price.append(item.find("span", class_="price-val").text.strip())
        area.append(item.find("ul", class_="name").text.strip())
        time.append(item.find("div", class_="city_when").text.strip())
        bill_of_sale.append(item.find("div", class_="bill_of_sale").text.strip() if item.find("div", class_="bill_of_sale") else "Çıxarış yoxdur")
        mortgage.append(item.find("div", class_="mortgage").text.strip() if item.find("div", class_="mortgage") else "Ipoteka yoxdur")
        repair.append(item.find("div", class_="repair").text.strip() if item.find("div", class_="repair") else "Təmirsiz")
data['location'] =location
data['price'] = price
data['area'] = area
data['time'] = time
data['bill_of_sale'] = bill_of_sale
data['mortgage'] = mortgage
data['repair'] = repair

In [27]:
data = pd.DataFrame(data)

In [28]:
data.head()

,location,price,area,time,bill_of_sale,mortgage,repair
0,Nərimanov r.,410 000,4 otaqlı200 m²2/14 mərtəbə,"Bakı, 23 avqust 2023",Çıxarış var,Ipoteka yoxdur,Təmirli
1,Gənclik m.,185 000,2 otaqlı61 m²3/17 mərtəbə,"Bakı, 02 sentyabr 2023",Çıxarış yoxdur,Ipoteka yoxdur,Təmirli
2,Nərimanov r.,418 404,5 otaqlı199.2 m²12/14 mərtəbə,"Bakı, bugün 12:33",Çıxarış yoxdur,Ipoteka yoxdur,Təmirsiz
3,Nəriman Nərimanov m.,159 000,2 otaqlı55 m²5/17 mərtəbə,"Bakı, 01 sentyabr 2023",Çıxarış var,İpoteka var,Təmirli
4,Koroğlu m.,269 000,3 otaqlı172 m²15/18 mərtəbə,"Bakı, bugün 21:53",Çıxarış var,Ipoteka yoxdur,Təmirli


In [29]:
data['price'] = data['price'].str.replace(' ', '').astype('int')

In [30]:
data['area_m2'] = data['area'].str.split('otaqlı').apply(lambda row: row[-1].split('m²')[0]).astype('float')

In [31]:
data['number_of_room'] = data['area'].str.split('otaqlı').apply(lambda row: int(row[0]) if len(row)==2 else np.nan)

In [32]:
data['floor'] = data['area'].str.split('otaqlı').apply(lambda row: int(row[-1].split('m²')[-1].split('/')[0])if 'mərtəbə' in row[-1] else np.nan)

In [33]:
data.drop('area', axis=1, inplace=True)

In [34]:
data.head()

,location,price,time,bill_of_sale,mortgage,repair,area_m2,number_of_room,floor
0,Nərimanov r.,410000,"Bakı, 23 avqust 2023",Çıxarış var,Ipoteka yoxdur,Təmirli,200.0,4,2
1,Gənclik m.,185000,"Bakı, 02 sentyabr 2023",Çıxarış yoxdur,Ipoteka yoxdur,Təmirli,61.0,2,3
2,Nərimanov r.,418404,"Bakı, bugün 12:33",Çıxarış yoxdur,Ipoteka yoxdur,Təmirsiz,199.2,5,12
3,Nəriman Nərimanov m.,159000,"Bakı, 01 sentyabr 2023",Çıxarış var,İpoteka var,Təmirli,55.0,2,5
4,Koroğlu m.,269000,"Bakı, bugün 21:53",Çıxarış var,Ipoteka yoxdur,Təmirli,172.0,3,15


In [35]:
data['date'] = data['time'].str.extract(r'(\d{2} [a-zA-Z]+ \d{4})')

In [36]:
data['date'] = data['date'].fillna("05 sentyabr 2023")
data.drop('time', axis=1, inplace=True)

In [37]:
data.head()

,location,price,bill_of_sale,mortgage,repair,area_m2,number_of_room,floor,date
0,Nərimanov r.,410000,Çıxarış var,Ipoteka yoxdur,Təmirli,200.0,4,2,23 avqust 2023
1,Gənclik m.,185000,Çıxarış yoxdur,Ipoteka yoxdur,Təmirli,61.0,2,3,02 sentyabr 2023
2,Nərimanov r.,418404,Çıxarış yoxdur,Ipoteka yoxdur,Təmirsiz,199.2,5,12,05 sentyabr 2023
3,Nəriman Nərimanov m.,159000,Çıxarış var,İpoteka var,Təmirli,55.0,2,5,01 sentyabr 2023
4,Koroğlu m.,269000,Çıxarış var,Ipoteka yoxdur,Təmirli,172.0,3,15,05 sentyabr 2023


In [38]:
data.to_csv('Task-Dirtydata.csv', index=False)